<a href="https://colab.research.google.com/github/innovativenexusbd/AgroAI/blob/main/nutrientprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

Importing the dataset, preparing and splitting the data

In [13]:
nutrient_df = pd.read_csv('synthetic_nutrient_data_10years.csv')
X = nutrient_df.drop(['label', 'season', 'year'], axis=1)
y = nutrient_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Initializing KNN and fitting the model

In [14]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

Predicting the Model

In [15]:
test_predictions = knn.predict(X_test)
prediction_df = X_test.copy()
prediction_df['predicted_label'] = test_predictions
prediction_df.to_csv('nutrient_predictions.csv', index=False)
print(classification_report(y_test, test_predictions))
label_counts = pd.Series(test_predictions).value_counts()
total_predictions = len(test_predictions)
comment = f"""
Nutrient Quality Prediction Summary:
- Good: {label_counts.get('good', 0)} ({(label_counts.get('good', 0) / total_predictions) * 100:.2f}%)
- Mid-good: {label_counts.get('mid-good', 0)} ({(label_counts.get('mid-good', 0) / total_predictions) * 100:.2f}%)
- Average: {label_counts.get('average', 0)} ({(label_counts.get('average', 0) / total_predictions) * 100:.2f}%)
- Mid-bad: {label_counts.get('mid-bad', 0)} ({(label_counts.get('mid-bad', 0) / total_predictions) * 100:.2f}%)
- Bad: {label_counts.get('bad', 0)} ({(label_counts.get('bad', 0) / total_predictions) * 100:.2f}%)
"""
if label_counts.get('bad', 0) > 0.5 * total_predictions:
    ultimate_prediction = "Overall Nutrient Quality: Bad"
elif label_counts.get('mid-bad', 0) > 0.5 * total_predictions:
    ultimate_prediction = "Overall Nutrient Quality: Mid-Bad"
elif label_counts.get('average', 0) > 0.5 * total_predictions:
    ultimate_prediction = "Overall Nutrient Quality: Average"
elif label_counts.get('mid-good', 0) > 0.5 * total_predictions:
    ultimate_prediction = "Overall Nutrient Quality: Mid-Good"
else:
    ultimate_prediction = "Overall Nutrient Quality: Good"

comment += ultimate_prediction
print(comment)
ultimate_df = pd.DataFrame({'ultimate_prediction': [ultimate_prediction]})

              precision    recall  f1-score   support

     average       0.29      0.22      0.25         9
         bad       0.25      0.40      0.31         5
        good       0.00      0.00      0.00         4
     mid-bad       0.40      0.18      0.25        11
    mid-good       0.20      0.33      0.25         3

    accuracy                           0.22        32
   macro avg       0.23      0.23      0.21        32
weighted avg       0.28      0.22      0.23        32


Nutrient Quality Prediction Summary:
- Good: 7 (21.88%)
- Mid-good: 5 (15.62%)
- Average: 7 (21.88%)
- Mid-bad: 5 (15.62%)
- Bad: 8 (25.00%)
Overall Nutrient Quality: Good


Saving the prediction

In [16]:
ultimate_df.to_csv('ultimate_nutrient_prediction.csv', index=False)